In [ ]:
import syft as sy
sy.requires(">=0.8.1-beta")
sy.enable_autoreload()

In [ ]:
domain_1 = sy.orchestra.launch(name="test-domain-1", dev_mode=True, reset=True)

In [ ]:
domain_1_client = domain_1.login(email="info@openmined.org", password="changethis")

In [ ]:
# domain_1_client = sy.login(port=8080, email="info@openmined.org", password="changethis")

In [ ]:
domain_2 = sy.orchestra.launch(name="test-domain-2", dev_mode=True, reset=True)

In [ ]:
domain_2_client = domain_2.login(email="info@openmined.org", password="changethis")

In [ ]:
# domain_2_client = sy.login(port=8081, email="info@openmined.org", password="changethis")

In [ ]:
domain_1_client.exchange_route(domain_2_client)

In [ ]:
domain_1_client.domains

In [ ]:
assert len(domain_1_client.domains) == 1

In [ ]:
domain_2_client.domains

In [ ]:
assert len(domain_2_client.domains) == 1

In [ ]:
from syft.service.project.project import NewProjectSubmit
from syft.service.project.project import NewProject
from syft.service.project.project import DemocraticConsensusModel
from syft.service.metadata.node_metadata import NodeMetadata

In [ ]:
project = NewProjectSubmit(
    name="test project",
    description="my test project",
    shareholders=[domain_1_client, domain_2_client],
    consensus_model=DemocraticConsensusModel(),
)
project

In [ ]:
project_id = sy.UID()
project_id

In [ ]:
p = domain_1_client.api.services.newproject.create_project(project, project_id)
p

In [ ]:
assert isinstance(p, NewProject)

In [ ]:
if domain_1.node_type.value == "python":
    domain_1.land()
domain_2.land()